In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.colors import to_rgb

# --- Parametri fissi ---
n_gif = 20
n_neurons = 500
duration_s = 30
bin_size_s = 0.05
n_bins = int(duration_s / bin_size_s)
window_s = 5
window_bins = int(window_s / bin_size_s)
step_bins = 1

log_median = np.log(0.001)
sigma = 0.6
p_stay_quiet = 0.95
p_stay_burst = 0.9
burst_rate_factor = 50
p_follow = 0.95

color_bottom = np.array(to_rgb("white"))
color_top = np.array(to_rgb("#666666"))  # grigio scuro

for n in range(n_gif):
    # --- Simulazione nuova per ogni GIF ---
    firing_rates = np.random.lognormal(mean=log_median, sigma=sigma, size=n_neurons)
    burst_states = np.zeros(n_bins, dtype=np.uint8)
    state = 0
    for t in range(n_bins):
        if state == 0 and np.random.rand() > p_stay_quiet:
            state = 1
        elif state == 1 and np.random.rand() > p_stay_burst:
            state = 0
        burst_states[t] = state

    neuronal_burst = np.zeros((n_neurons, n_bins), dtype=np.uint8)
    for i in range(n_neurons):
        follow = np.random.rand(n_bins) < p_follow
        neuronal_burst[i] = np.where(follow, burst_states, 1 - burst_states)

    spikes = np.zeros((n_neurons, n_bins), dtype=np.uint8)
    for i, fr in enumerate(firing_rates):
        for t in range(n_bins):
            state = neuronal_burst[i, t]
            rate = fr if state == 0 else fr * burst_rate_factor
            p_spike = rate * bin_size_s
            spikes[i, t] = np.random.rand() < p_spike

    neuron_ids, spike_bins = np.where(spikes)

    # --- Gradient colore neuroni (bianco->grigio) ---
    gradient = np.linspace(0, 1, n_neurons)[:, None]
    neuron_colors = color_top * (1 - gradient) + color_bottom * gradient

    fig, ax = plt.subplots(figsize=(10, 6))
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    def update(frame):
        ax.clear()
        fig.patch.set_facecolor('black')
        ax.set_facecolor('black')
        start_bin = frame
        end_bin = start_bin + window_bins
        mask = (spike_bins >= start_bin) & (spike_bins < end_bin)
        these_bins = spike_bins[mask] - start_bin
        these_neurons = neuron_ids[mask]
        if len(these_neurons) > 0:
            ax.scatter(these_bins * bin_size_s, these_neurons,
                       s=30, c=neuron_colors[these_neurons], marker='o', alpha=1, linewidths=0)
        ax.set_xlim(0, window_s)
        ax.set_ylim(-1, n_neurons)
        ax.axis('off')
        plt.tight_layout(pad=0)

    n_frames = spikes.shape[1] - window_bins
    ani = FuncAnimation(fig, update, frames=n_frames, interval=50, blit=False)
    ani.save(f"neural_activity_gradient_{n+1:02d}.gif", writer=PillowWriter(fps=10))
    plt.close(fig)
    print(f"Saved GIF {n+1}/{n_gif}")

print("Fatto!")


Saved GIF 1/20
